# Setting

##  Imports and Setup

Let's start with the base imports.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# This Colab requires TF 2.5.
!pip install -U "tensorflow>=2.5"

In [ ]:
import os
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import shutil
import cv2
import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont, ImageOps
from six.moves.urllib.request import urlopen
import tensorflow as tf
import tensorflow_hub as hub
import math
import glob

tf.get_logger().setLevel('ERROR')

In [ ]:
# @title  import models and dats set keypoints

def load_image_into_numpy_array(path):
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))

  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)


ALL_MODELS = {

'Faster R-CNN ResNet50 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1',
'Faster R-CNN ResNet50 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_1024x1024/1',
'Faster R-CNN ResNet50 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_800x1333/1',
'Faster R-CNN ResNet101 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_640x640/1',
'Faster R-CNN ResNet101 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_1024x1024/1',
'Faster R-CNN ResNet101 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_800x1333/1',
'Faster R-CNN ResNet152 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_640x640/1',
'Faster R-CNN ResNet152 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_1024x1024/1',
'Faster R-CNN ResNet152 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_800x1333/1',
'Faster R-CNN Inception ResNet V2 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_640x640/1',
'Faster R-CNN Inception ResNet V2 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_1024x1024/1',
'Mask R-CNN Inception ResNet V2 1024x1024' : 'https://tfhub.dev/tensorflow/mask_rcnn/inception_resnet_v2_1024x1024/1'
}

#List of tuples with Human Keypoints for the COCO 2017 dataset. This is needed for models with keypoints.

COCO17_HUMAN_POSE_KEYPOINTS = [(0, 1),
 (0, 2),
 (1, 3),
 (2, 4),
 (0, 5),
 (0, 6),
 (5, 7),
 (7, 9),
 (6, 8),
 (8, 10),
 (5, 6),
 (5, 11),
 (6, 12),
 (11, 12),
 (11, 13),
 (13, 15),
 (12, 14),
 (14, 16)]

## Visualization tools

To visualize the images with the proper detected boxes, keypoints and segmentation, we will use the TensorFlow Object Detection API. To install it we will clone the repo.

In [ ]:
# @title  Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# @title  Intalling the Object Detection API

#a cell magic command that allows you to run multiple shell commands as a script
%%bash 
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .


In [ ]:
# @title  Import the dependencies will be needed later

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

Load label map data from the repository that we loaded the Object Detection API code

In [ ]:
#  @title  getting label map
# Set the path to the directory containing the label map
path_to_labelmap = './models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(path_to_labelmap, use_display_name=True)

## Build a detection model and load pre-trained model weights

Here we will choose which Object Detection model we will use. Select the architecture and it will be loaded automatically. 

In [ ]:
model_display_name = 'Faster R-CNN ResNet50 V1 640x640' # @param ['CenterNet HourGlass104 512x512','CenterNet HourGlass104 Keypoints 512x512','CenterNet HourGlass104 1024x1024','CenterNet HourGlass104 Keypoints 1024x1024','CenterNet Resnet50 V1 FPN 512x512','CenterNet Resnet50 V1 FPN Keypoints 512x512','CenterNet Resnet101 V1 FPN 512x512','CenterNet Resnet50 V2 512x512','CenterNet Resnet50 V2 Keypoints 512x512','EfficientDet D0 512x512','EfficientDet D1 640x640','EfficientDet D2 768x768','EfficientDet D3 896x896','EfficientDet D4 1024x1024','EfficientDet D5 1280x1280','EfficientDet D6 1280x1280','EfficientDet D7 1536x1536','SSD MobileNet v2 320x320','SSD MobileNet V1 FPN 640x640','SSD MobileNet V2 FPNLite 320x320','SSD MobileNet V2 FPNLite 640x640','SSD ResNet50 V1 FPN 640x640 (RetinaNet50)','SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)','SSD ResNet101 V1 FPN 640x640 (RetinaNet101)','SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)','SSD ResNet152 V1 FPN 640x640 (RetinaNet152)','SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)','Faster R-CNN ResNet50 V1 640x640','Faster R-CNN ResNet50 V1 1024x1024','Faster R-CNN ResNet50 V1 800x1333','Faster R-CNN ResNet101 V1 640x640','Faster R-CNN ResNet101 V1 1024x1024','Faster R-CNN ResNet101 V1 800x1333','Faster R-CNN ResNet152 V1 640x640','Faster R-CNN ResNet152 V1 1024x1024','Faster R-CNN ResNet152 V1 800x1333','Faster R-CNN Inception ResNet V2 640x640','Faster R-CNN Inception ResNet V2 1024x1024','Mask R-CNN Inception ResNet V2 1024x1024']
model_handle = ALL_MODELS[model_display_name]

print('Selected model:'+ model_display_name)
print('Model Handle at TensorFlow Hub: {}'.format(model_handle))

## Loading the model


In [ ]:
try:
    model = hub.load(model_handle)
    print('model loaded!')
except:
    print('model failed to load')

# Sepration of human and non human images

In [ ]:
# @title Classify, create new folders in existig folders ad sed the images to the new folders

#Load the label map
category_index = {1: {'id': 1, 'name': 'person'}}

# Define the path to the directory containing the images
original_dir = '/content/drive/MyDrive/HAZIQ/imagejpeg'

# Create two new directories for human and no_human images
human_dir = os.path.join(original_dir, 'Human')
os.makedirs(human_dir, exist_ok=True)
no_human_dir = os.path.join(original_dir, 'No human')
os.makedirs(no_human_dir, exist_ok=True)

# Loop through each file and load images that have valid image extensions
for file in os.listdir(original_dir):
    # Exclude directories and non-image files
    if not os.path.isdir(os.path.join(original_dir, file)) and \
       os.path.splitext(file)[1].lower() in ['.jpg', '.jpeg', '.png', '.bmp']:
        
        # Load the image into a PIL Image object
        image_pil = Image.open(os.path.join(original_dir, file))

        # Convert the image to a numpy array
        image_np = np.array(image_pil)

        # Expand the dimensions of the array to make it compatible with the model
        image_np = np.expand_dims(image_np, axis=0)
        
        # Running inference
        results = model(image_np)

        # Convert the images to NumPy array
        result = {key:value.numpy() for key,value in results.items()}

        # Filter for person class only
        person_mask = (result['detection_classes'][0] == 1)

        if person_mask.any():  # Check if at least one person is detected
            # Save image to human_dir
            shutil.move(os.path.join(original_dir, file), os.path.join(human_dir, file))
        else:
            # Save image to no_human_dir
            shutil.move(os.path.join(original_dir, file), os.path.join(no_human_dir, file))

In [ ]:
# @title Show the images with human in it

# Resizes an image to the specified size and pads the image
def resize_and_pad(image, size, pad_color):
    # Resize the image while maintaining its aspect ratio
    original_size = max(image.size)
    new_size = (int(size * image.size[0] / original_size), int(size * image.size[1] / original_size))
    image_resized = image.resize(new_size, Image.BICUBIC)

    # Create a new image of the desired size with the given color
    new_image = Image.new("RGB", (size, size), pad_color)

    # Paste the resized image onto the new image, centered
    x_offset = (size - new_size[0]) // 2
    y_offset = (size - new_size[1]) // 2
    new_image.paste(image_resized, (x_offset, y_offset))

    return new_image

# Define the input size
input_size = 640

# Define the number of columns per row
columns = 3

# Check if there are any images in the directory, if not, skip the loop
if len(os.listdir(human_dir)) == 0:
    print('No images with human found in the directory!')
else:
    num_images = len(os.listdir(human_dir))
    rows = num_images // columns
    if num_images % columns != 0:
        rows += 1

    # Creates a grid of subplots
    fig, axes = plt.subplots(rows, columns, figsize=(15, rows*5))

    # Adjusts the spacing between subplots
    fig.subplots_adjust(hspace=0.1, wspace=0.1)

    # Convert axes to a 2D array if it is 1D
    if rows == 1 or columns == 1:
        axes = np.array(axes).reshape(rows, columns)
    
    # Loop through each file in the human directory and display the image
    for i, file in enumerate(os.listdir(human_dir)):

        # Load the image into a PIL Image object
        image_pil = Image.open(os.path.join(human_dir, file))

        # Calculate the maximum dimension of the image
        max_dimension = max(image_pil.size)

        # Resize and pad the image to make it square
        resized_image = resize_and_pad(image_pil, input_size, (0, 0, 0))

        # Calculate the row and column for this image
        row = i // columns
        col = i % columns

        # Add the image to the plot
        ax = axes[row, col]
        ax.imshow(resized_image)
        ax.axis('off')

        # Set aspect ratio to square
        ax.set_aspect('equal')
    
    # Adjust the subplot for each row
    for i in range(rows):
        for j in range(columns):
            if j >= num_images % columns and i == rows - 1:
                axes[i, j].axis('off')
                axes[i, j].set_xticklabels([])
                axes[i, j].set_yticklabels([])
    
    # Set the title of the plot
    fig.suptitle('Images with human', fontsize=24)
    plt.show()


In [ ]:
# @title Show the images ***without any human*** in it

# Resizes an image to the specified size and pads the image
def resize_and_pad(image, size, pad_color):

    # Resize the image while maintaining its aspect ratio
    original_size = max(image.size)
    new_size = (int(size * image.size[0] / original_size), int(size * image.size[1] / original_size))
    image_resized = image.resize(new_size, Image.BICUBIC)

    # Create a new image of the desired size with the given color
    new_image = Image.new("RGB", (size, size), pad_color)

    # Paste the resized image onto the new image, centered
    x_offset = (size - new_size[0]) // 2
    y_offset = (size - new_size[1]) // 2
    new_image.paste(image_resized, (x_offset, y_offset))

    return new_image

# Define the input size
input_size = 640

# Define the number of columns per row
columns = 3

# Check if there are any images in the directory, if not, skip the loop
if len(os.listdir(no_human_dir)) == 0:
    print('No images without human found in the directory!')
else:
    num_images = len(os.listdir(no_human_dir))
    rows = num_images // columns
    if num_images % columns != 0:
        rows += 1

    # Creates a grid of subplots
    fig, axes = plt.subplots(rows, columns, figsize=(15, rows*5))

    # Adjusts the spacing between subplots
    fig.subplots_adjust(hspace=0.1, wspace=0.1)

    # Convert axes to a 2D array if it is 1D
    if rows == 1 or columns == 1:
        axes = np.array(axes).reshape(rows, columns)
    
    # Loop through each file in the human directory and display the image
    for i, file in enumerate(os.listdir(no_human_dir)):

        # Load the image into a PIL Image object
        image_pil = Image.open(os.path.join(no_human_dir, file))

        # Calculate the maximum dimension of the image
        max_dimension = max(image_pil.size)

        # Resize and pad the image to make it square
        resized_image = resize_and_pad(image_pil, input_size, (0, 0, 0))

        # Calculate the row and column for this image
        row = i // columns
        col = i % columns

        # Add the image to the plot
        ax = axes[row, col]
        ax.imshow(resized_image)
        ax.axis('off')

        # Set aspect ratio to square
        ax.set_aspect('equal')
    
    # Adjust the subplot for each row
    for i in range(rows):
        for j in range(columns):
            if j >= num_images % columns and i == rows - 1:
                axes[i, j].axis('off')
                axes[i, j].set_xticklabels([])
                axes[i, j].set_yticklabels([])
    
    # Set the title of the plot
    fig.suptitle('Images without any Human', fontsize=24)
    plt.show()



# Create bounding boxes, crop and save as new images in new folder

In [ ]:
# @title Create bounding boxes

# Create the new directory
boxes_human_dir = os.path.join(original_dir, 'Human with bounding box')
os.makedirs(boxes_human_dir, exist_ok=True)

# Define the input size
input_size = 640

# Define the number of columns per row
columns = 3

# Check if there are any images in the directory, if not, skip the loop
num_images = len(os.listdir(human_dir))
rows = num_images // columns
if num_images % columns != 0:
    rows += 1

# Creates a grid of subplots
fig, axes = plt.subplots(rows, columns, figsize=(15, rows*5))

# Adjusts the spacing between subplots
fig.subplots_adjust(hspace=0.1, wspace=0.1)

# Convert axes to a 2D array if it is 1D
if rows == 1 or columns == 1:
    axes = np.array(axes).reshape(rows, columns)

# Define the function to load images into NumPy arrays
def load_image_into_numpy_array(path):
    with open(path, 'rb') as f:
        im = Image.open(f)
        return np.array(im)

# Load the images into an array.
image_paths = glob.glob(os.path.join(human_dir, '*.*'))
image_np_array = []
for path in image_paths:
    image_np = load_image_into_numpy_array(path)
    image_np_array.append(image_np)

# Running inference and visualizing the results for all images
for i, image_np in enumerate(image_np_array):

    # Convert the NumPy array to a tensor with the required shape and dtype
    input_tensor = tf.convert_to_tensor(image_np, dtype=tf.uint8)
    input_tensor = tf.expand_dims(input_tensor, 0)

    # Run inference on the input tensor
    results = model(input_tensor)

    # Convert the values to numpy arrays
    result = {key:value.numpy() for key,value in results.items()}

    # Get the indices of the detected human objects
    human_indices = np.where(result['detection_classes'][0] == 1)[0]

    label_id_offset = 0
    image_np_with_detections = image_np.copy()

    # Use keypoints if available in detections
    keypoints, keypoint_scores = None, None
    if 'detection_keypoints' in result:
      keypoints = result['detection_keypoints'][0]
      keypoint_scores = result['detection_keypoint_scores'][0]

    # Draw bounding boxes
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          result['detection_boxes'][0, human_indices],
          (result['detection_classes'][0, human_indices] + label_id_offset).astype(int),
          result['detection_scores'][0, human_indices],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.5, # To filter out detections with scores below the specified threshold. 
          #Detections with scores below this threshold are considered false positives and are not shown in the output.
          #The value is adjustable to a different threshold if you want to see more or fewer detections in the output.
          agnostic_mode=False,
          skip_labels=True) # To remove class name label
    
    # Save the image with bounding boxes in the new directory
    img_name = os.path.basename(image_paths[i])
    img_path = os.path.join(boxes_human_dir, img_name)
    resized_image.save(img_path)

    # Resize and pad the image to make it square
    resized_image = resize_and_pad(image_pil, input_size, (0, 0, 0)) 

    # Convert the NumPy array to a PIL Image object
    img = Image.fromarray(image_np_with_detections)

    # Resize and pad the image to make it square
    resized_image = resize_and_pad(img, input_size, (0, 0, 0))

    # Calculate the row and column for this image
    row = i // columns
    col = i % columns

    # Add the image to the plot
    ax = axes[row, col]
    ax.imshow(resized_image)
    ax.axis('off')

    # Set aspect ratio to square
    ax.set_aspect('equal')
        
# Adjust the subplot for each row
for i in range(rows):
    for j in range(columns):
        if j >= num_images % columns and i == rows - 1:
            axes[i, j].axis('off')
            axes[i, j].set_xticklabels([])
            axes[i, j].set_yticklabels([])
        
# Set the title of the plot
plt.show()

In [ ]:
# @title Crop the images according to bounding boxes and save it in new folder

# Get a list of all image files in the input directory
image_files = [file for file in os.listdir(human_dir) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Create a new directory for cropped images
cropped_dir = os.path.join(original_dir, 'Cropped images')
os.makedirs(cropped_dir, exist_ok=True)

target_size = 256  # Set the desired size for the cropped images
detection_threshold = 0.5  # Set the detection threshold for displaying bounding boxes

# Resizes an image to the specified size and pads the image
def resize_and_pad_image(image_np, size):
    height, width = image_np.shape[:2]

    # Resize the image
    if height > width:
        scale = size / height
    else:
        scale = size / width
    image_np = cv2.resize(image_np, (int(width * scale), int(height * scale)))

    # Pad the image to make it square
    max_dim = max(image_np.shape[:2])
    top = bottom = (max_dim - image_np.shape[0]) // 2
    left = right = (max_dim - image_np.shape[1]) // 2
    image_np = cv2.copyMakeBorder(image_np, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))

    return image_np

# Running inference and visualizing the results for all images.
for image_path in image_files:
    # Load the image
    image_np = cv2.imread(os.path.join(human_dir, image_path))

    # Convert the NumPy array to a tensor with the required shape and dtype.
    input_tensor = tf.convert_to_tensor(image_np, dtype=tf.uint8)
    input_tensor = tf.expand_dims(input_tensor, 0)

    # Run inference on the input tensor.
    results = model(input_tensor)

    # Change to NumPy array
    result = {key: value.numpy() for key, value in results.items()}

    # Get the indices of the detected human objects.
    human_indices = np.where(result['detection_classes'][0] == 1)[0]

    # Initialize the list of cropped images.
    cropped_images = []

    # Iterate over all detected humans and crop the corresponding bounding boxes.
    for idx in human_indices:
        # Check if the bounding box corresponds to a human (class label 1) and passes the threshold.
        if result['detection_scores'][0, idx] >= detection_threshold:
            ymin, xmin, ymax, xmax = result['detection_boxes'][0, idx]

            # Convert the box coordinates from normalized form to pixel values.
            im_height, im_width, _ = image_np.shape
            ymin = int(ymin * im_height)
            xmin = int(xmin * im_width)
            ymax = int(ymax * im_height)
            xmax = int(xmax * im_width)

            # Draw the bounding box on the original image.
            cv2.rectangle(image_np, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

            # Crop the image using the bounding box coordinates.
            cropped_image = image_np[ymin:ymax, xmin:xmax, :]

            # Resize and pad the cropped image to the target size.
            image_padded = resize_and_pad_image(cropped_image, target_size)

            # Save the cropped image in the new directory
            img_name = os.path.basename(image_path)
            img_name = os.path.splitext(img_name)[0]
            cropped_image_path = os.path.join(cropped_dir, f"{img_name}cropped{idx}.jpg")
            cv2.imwrite(cropped_image_path, image_padded)

            # Append the cropped image to the list of cropped images
            cropped_images.append(image_padded)

    # Visualize the original image with bounding boxes and cropped images
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    plt.subplots_adjust(hspace=0.1, wspace=0.1)

    # Calculate the padding size for making the image square
    im_height, im_width, _ = image_np.shape
    max_dim = max(im_height, im_width)
    pad_top = (max_dim - im_height) // 2
    pad_bottom = max_dim - im_height - pad_top
    pad_left = (max_dim - im_width) // 2
    pad_right = max_dim - im_width - pad_left

    # Add padding to the original image
    padded_image = cv2.copyMakeBorder(image_np, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=(0, 0, 0))

# Show the padded image with bounding boxes
    axes[0].imshow(cv2.cvtColor(padded_image, cv2.COLOR_BGR2RGB))
    axes[0].axis('off')

    # Add empty subplots for spacing
    axes[1].axis('off')
    axes[2].axis('off')

    # Draw bounding boxes on the padded image
    for idx in human_indices:
        if result['detection_scores'][0, idx] >= detection_threshold:
            ymin, xmin, ymax, xmax = result['detection_boxes'][0, idx]
            ymin = int(ymin * im_height) + pad_top
            xmin = int(xmin * im_width) + pad_left
            ymax = int(ymax * im_height) + pad_top
            xmax = int(xmax * im_width) + pad_left
            cv2.rectangle(padded_image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

    # Show the padded image with bounding boxes
    axes[0].imshow(cv2.cvtColor(padded_image, cv2.COLOR_BGR2RGB))

    fig.suptitle('Original image', fontsize=20, ha='right')
    plt.show()

    # Display the cropped images.
    num_cropped_images = len(cropped_images)
    if num_cropped_images > 0:
        columns = 3
        rows = (num_cropped_images + 3 - 1) // columns
        fig, axes = plt.subplots(rows, columns, figsize=(15, rows * 5))
        fig.suptitle('Cropped images', fontsize=20, ha='right')
        plt.subplots_adjust(hspace=0.1, wspace=0.1)

        for i, img in enumerate(cropped_images):
            axes = axes.reshape(rows, 3)
            row_idx = i // columns
            col_idx = i % columns
            ax = axes[row_idx, col_idx]
            ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB
            ax.axis('off')

        # Add empty subplots to the last row if necessary.
        if num_cropped_images % columns != 0:
            num_empty_subplots = columns - (num_cropped_images % columns)
            for i in range(num_empty_subplots):
                row_idx = rows - 1
                col_idx = num_cropped_images % columns + i
                ax = axes[row_idx, col_idx]
                ax.axis('off')

        # Add empty subplots to the last row if necessary.
        if num_cropped_images % columns != 0:
            num_empty_subplots = columns - (num_cropped_images % columns)
            for i in range(num_empty_subplots):
                row_idx = rows - 1
                col_idx = num_cropped_images % columns + i
                ax = axes[row_idx, col_idx]
                ax.axis('off')

        plt.show()

# Training new model for classification

In [ ]:
!pip install keras
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# Import the libraries as shown below
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from glob import glob

# Re-size all the images
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/HAZIQ/data set/images/train'
valid_path = '/content/drive/MyDrive/HAZIQ/data set/images/val'

resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# Freeze the layers of the VGG16 model
for layer in resnet.layers:
    layer.trainable = False

# useful for getting number of output classes
folders = glob('/content/drive/MyDrive/HAZIQ/data set/images/train/*')

# Our layers
x = Flatten()(resnet.output)
x = Dense(512, activation='relu')(x)  # Number of neurons in the dense layer
prediction = Dense(len(folders), activation='softmax')(x)

# Create  model object
model = Model(inputs=resnet.input, outputs=prediction)

# view the structure of the model
model.summary()

# Compile the model
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Generate training and validation data
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/HAZIQ/data set/images/train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/HAZIQ/data set/images/val',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

In [ ]:
# Train the model with the modified settings
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=2,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# @title Save the model
model.save('vgg_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('vgg_model.h5')


# Detecting the pose using new model

In [ ]:
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model('vgg_model.h5')

# List all the image files in the directory
image_files = os.listdir(cropped_dir)

# Define the class weights
class_weights = {0: 1.0, 1: 1.5}  # Adjust the weights as per your requirement

# Define the class labels
class_labels = ['Fall', 'Sit', 'Walk']

# Calculate the number of rows and columns for subplots
num_images = len(image_files)
num_rows = (num_images + 2) // 3  # Add 2 to account for rounding up
num_cols = min(num_images, 3)

# Create a figure and subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 4*num_rows))

# Iterate over the image files
for i, image_file in enumerate(image_files):
    # Load and preprocess the image
    image_path = os.path.join(cropped_dir, image_file)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert image to RGB
    img = cv2.resize(img, (224, 224))  # Resize the image to match the expected input size of your model
    img = img / 255.0  # Normalize pixel values between 0 and 1
    img = np.expand_dims(img, axis=0)

    # Make predictions on the image using your trained model
    predictions = model.predict(img)

    # Get the predicted class probabilities
    probabilities = predictions[0]

    # Calculate the accuracy rates for each class
    accuracy_rates = probabilities * 100

    # Get the predicted label and accuracy rate with the highest accuracy
    highest_accuracy_idx = np.argmax(accuracy_rates)
    highest_label = class_labels[highest_accuracy_idx]
    highest_accuracy = accuracy_rates[highest_accuracy_idx]

    # Display the input image with the predicted label and highest accuracy rate in the corresponding subplot
    row_idx = i // num_cols
    col_idx = i % num_cols
    axes[row_idx, col_idx].imshow(img[0])
    axes[row_idx, col_idx].axis('off')
    axes[row_idx, col_idx].set_title(f"{highest_label}: {highest_accuracy:.2f}%", color=(1.0, 0.0, 0.0))

# Remove empty subplots if any
for i in range(num_images, num_rows * num_cols):
    ax = axes.flatten()[i]
    ax.axis('off')

# Show the figure
plt.tight_layout()
plt.show()


# Detecting the image using GUI

In [ ]:
from tensorflow.keras.models import load_model
from google.colab import files
from IPython.display import display, clear_output
import ipywidgets as widgets

# Load your trained model
model = load_model('vgg_model.h5')

# Define the class labels
class_labels = ['Fall', 'Sit', 'Walk']

# Function to perform predictions on the selected image
def predict_image():
    # Upload an image file
    uploaded = files.upload()
    file_path = next(iter(uploaded))

    # Load and preprocess the image
    img = cv2.imread(file_path)
    img = cv2.resize(img, (224, 224))  # Resize the image to match the expected input size of your model
    img = img / 255.0  # Normalize pixel values between 0 and 1
    img = np.expand_dims(img, axis=0)

    # Convert the image to a compatible depth
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

    # Make predictions on the image using your trained model
    predictions = model.predict(img)

    # Make predictions on the image using your trained model
    predictions = model.predict(img)

    # Get the predicted class probabilities
    probabilities = predictions[0]

    # Calculate the accuracy rates for each class
    accuracy_rates = probabilities * 100

    # Get the predicted label and accuracy rate with the highest accuracy
    highest_accuracy_idx = np.argmax(accuracy_rates)
    highest_label = class_labels[highest_accuracy_idx]
    highest_accuracy = accuracy_rates[highest_accuracy_idx]

    # Display the image and the label with the highest accuracy
    plt.imshow(cv2.cvtColor(img[0], cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(f'{highest_label}: {highest_accuracy:.2f}%')
    plt.show()

    # Display the reupload button
    button = widgets.Button(description="Reupload Image")
    display(button)

    # Function to handle button click event
    def reupload_image(button):
        # Clear the previous output
        clear_output()

        # Call the predict_image function again
        predict_image()

    # Attach the button click event
    button.on_click(reupload_image)

# Call the function to perform predictions
predict_image()
